# COMBINED DEAM 2000+PM

In [1]:
import pandas as pd
import os
import numpy as np
from math import sqrt

from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
# from nltk.stem.snowball import SnowballStemmer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD

In [3]:
FEATURES = 'D:\\Study\\PMEmo2019\\PMEmo2019\\features\\'
ANNOTATIONS = 'C:\\Users\\asus\\Desktop'
DATASET_DIR_2 = 'D:\\Study\\PMEmo2019\\'


In [4]:
def load_static_features():
    features_csv = os.path.join(FEATURES, 'combined_2000.csv')
    static_features= pd.read_csv(features_csv, index_col=0)
    return static_features

def load_static_features_and_valence():
    static_features = load_static_features()
    valence_csv = os.path.join(ANNOTATIONS, 'FINAL_AV.csv')
    valence = pd.read_csv(valence_csv, index_col=0, usecols=['musicId','Valence'])
    return static_features.join(valence).dropna()

def load_static_features_and_arousal():
    static_features = load_static_features()
    arousal_csv = os.path.join(ANNOTATIONS, 'FINAL_AV.csv')
    arousal = pd.read_csv(arousal_csv, index_col=0, usecols=['musicId','Arousal'])
    return static_features.join(arousal).dropna()

def load_audio_dataset(data):
    features = data[data.columns[:-1]].values
    labels = data[data.columns[-1]].values
#     scaler = StandardScaler(copy=False)
#     scaler.fit_transform(features)
    return features, labels

def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))


regressors = {
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'Ridge': Ridge(),
    'kNN': KNeighborsRegressor(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    'SVRpoly': SVR(kernel='poly', gamma='scale'),
    'SVRlinear': SVR(kernel='linear', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1),
    'MLP': MLPRegressor(hidden_layer_sizes=(200,50), max_iter=2000),
    'AdaBoost': AdaBoostRegressor(n_estimators=10),
}

from tqdm import notebook
import IPython.display as ipd


def cross_val_regression(regressors, features, labels, preprocessfunc):
    columns = list(regressors.keys())
    scores = pd.DataFrame(columns=columns, index=['RMSE'])

    for reg_name, reg in notebook.tqdm(regressors.items(), desc='regressors'):
        scorer = {'rmse': make_scorer(rmse)}
        reg = make_pipeline(*preprocessfunc, reg)
        
        reg_score = cross_validate(reg, features, labels, scoring=scorer, cv=5, return_train_score=False)
        scores.loc['RMSE', reg_name] = reg_score['test_rmse'].mean()
    #         scores.loc['R', reg_name] = reg_score['test_r'].mean()

    mean_rmse = scores.mean(axis=1)
    std_rmse = scores.std(axis=1)

    scores['Mean'] = mean_rmse
    scores['std'] = std_rmse
    return scores


def format_scores(scores):
    def highlight(s):
        is_min = s == min(s)
        #         is_max = s == max(s)
        #         is_max_or_min = (is_min | is_max)
        return ['background-color: yellow' if v else '' for v in is_min]

    scores = scores.style.apply(highlight, axis=1, subset=pd.IndexSlice[:, :scores.columns[-2]])
    return scores.format('{:.3f}')


prefunc = [StandardScaler()]

print('In Arousal dimension...')
data_a = load_static_features_and_arousal()
features_a, labels_a = load_audio_dataset(data_a)

scores_a_a = cross_val_regression(regressors, features_a, labels_a, prefunc)
ipd.display(format_scores(scores_a_a))



print('In Valence dimension...')
data_v = load_static_features_and_valence()
features_v, labels_v = load_audio_dataset(data_v)

scores_a_v = cross_val_regression(regressors, features_v, labels_v, prefunc)
ipd.display(format_scores(scores_a_v))

In Arousal dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,Mean,std
RMSE,0.340,0.340,3.917,0.266,0.230,1250201.297,5.309,0.281,0.304,50.486,0.256,113660.275,376948.013


In Valence dimension...


,Lasso,ElasticNet,Ridge,kNN,SVRrbf,SVRpoly,SVRlinear,DT,RF,MLP,AdaBoost,Mean,std
RMSE,0.301,0.301,5.064,0.247,0.218,741397.729,3.645,0.260,0.268,30.499,0.228,67403.524,223538.589
